In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json
import seaborn as sns
import results_functions

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

### Make descriptive boxplots for all

In [ ]:
directory = os.path.join(json_path_onedrive,
    'results', 'Avg_Features')

reload(results_functions)
saving = 1
df_fu0m, df_fu3m, df_fu12m = results_functions.get_descriptives(directory, saving)

'''
with open(os.path.join(directory,
    'Means_FU0M.pkl'), "rb") as file:
    val_dat = pickle.load(file)
'''

### Make correlations with TEED

In [ ]:
directory_TEED = os.path.join(json_path_onedrive,
    'results', 'Stim_pars', 'TEED')


stim_teed = pd.read_csv(os.path.join(directory_TEED, filename_teed))


In [ ]:
directory_Feat = os.path.join(json_path_onedrive,
    'results', 'Avg_Features')

all_sums_dfs = pd.DataFrame()

for filename in os.listdir(directory_Feat):
    if filename.endswith("AvgFeatures.csv"):
        file_path = os.path.join(directory_Feat, filename)
        df = pd.read_csv(file_path, index_col=None)
        df = df.drop('Unnamed: 0', axis = 1)

        Sub_id = filename[0:6]

        fu12m_index = df[df['TimePoint'] == 'FU12M'].index[0]
        mask = df.index < fu12m_index
        sums_until_fu12m = df[mask].sum()

        this_sums = sums_until_fu12m.to_frame().transpose()
        this_sums = this_sums.drop(['FirstBatVal','LastBatVal'], axis = 1)
        this_sums.insert(0, 'SubID', Sub_id)
        this_sums['Battery_12mfu'] = df.loc[fu12m_index,'FirstBatVal']

        all_sums_dfs = pd.concat([all_sums_dfs, this_sums])

In [ ]:
directory_TEED = os.path.join(json_path_onedrive,
    'results', 'Stim_pars', 'TEED')

all_teeds_dfs = pd.DataFrame()

for filename in os.listdir(directory_TEED):
    if filename.endswith("TEDD.csv"):
        file_path = os.path.join(directory_TEED, filename)
        stim_teed = pd.read_csv(file_path, index_col=None)

        Total_teed = sum(stim_teed['TEDD'])

        this_teed = pd.DataFrame()
        this_teed['SubID'] = stim_teed['SubID'].unique()
        this_teed['TEED'] = Total_teed

        all_teeds_dfs = pd.concat([all_teeds_dfs, this_teed])

In [ ]:
corr_df = all_sums_dfs.merge(all_teeds_dfs, on = 'SubID')
corr_df

In [ ]:
plt.scatter(corr_df['Battery_12mfu'], corr_df['TEED'])